### Задание №3
### Объектно-Ориентированное Программирование

Задание состоит в том, чтобы реализовать базовые алгоритмы линейной алгебры с помощью базовых средств объектно-ориентированного программирования Python.

Для решения задачи требуется реализовать класс Matrix с базовыми алгоритмами линейной алгебры:

\_\_mul\_\_ - умножение матриц

\_\_add\_\_ - поэлементное сложение матриц

\_\_sub\_\_ - поэлементное вычитание матриц

\_\_pow\_\_ - возведение элементов матрицы в скаларную степень

transpose() - транспонирование матрицы

inverse() - обращение матрицы

det() - определитель матрицы

shape() - размер матрицы (кортеж)

sum() - сумма всех элементов матрицы

а также служебные методы:

\_\_getitem\_\_ - получить элемент по индексу

\_\_setitem\_\_ - задать элемент по индексу

\_\_repr\_\_ и \_\_str\_\_

### Отправка задания

Для сдачи задания необходимо отправить боту @py2021sharebot с указанием seminar03 два файла:
1. result.json
2. seminar03.ipynb

Автоматическая проверка отправки будет реализована командой /check seminar03.

In [1]:
import sys
import os
import math
import copy
import json
from numpy.linalg import inv
import random
import numpy as np

Требуется реализовать методы в следующем классе, отмеченные #TODO:

In [2]:
"""Модуль базовых алгоритмов линейной алгебры.
Задание состоит в том, чтобы имплементировать класс Matrix
(следует воспользоваться кодом из семинара ООП), учтя рекомендации pylint.
Для проверки кода следует использовать команду pylint matrix.py.
Pylint должен показывать 10 баллов.
Кроме того, следует добавить поддержку исключений в отмеченных местах.
Для проверки корректности алгоритмов следует сравнить результаты с соответствующими функциями numpy.
"""
import json
import random
import numpy as np
from numpy.linalg import inv, det


class Matrix:
    """
    Matrix class
    """
    def __init__(self,  nrows, ncols, init = "zeros"):
        """
            nrows - количество строк матрицы
            ncols - количество столбцов матрицы
            init - метод инициализации элементов матрицы:
                "zeros" - инициализация нулями
                "ones" - инициализация единицами
                "random" - случайная инициализация
                "eye" - матрица с единицами на главной диагонали
        """
        if nrows<0 or ncols<0:
            raise ValueError('nrows and ncols must by a positive number.')
        if init not in ["zeros", "ones", "eye", "random"]:
            raise ValueError(f'init has not atribute {init}')
        self.nrows = nrows
        self.ncols = ncols
        self.data = []
        if init == "zeros":
            for i in range(0,nrows):
                row = []
                for _ in range(0,ncols):
                    row.append(0)
                self.data.append(row)
        if init == "ones":
            for i in range(0,nrows):
                row = []
                for _ in range(0,ncols):
                    row.append(1)
                self.data.append(row)
        if init == "random":
            for i in range(0,nrows):
                row = []
                for _ in range(0,ncols):
                    row.append(random.randint(1,100))
                self.data.append(row)
        if init == "eye":
            for i in range(0,nrows):
                row = []
                for _ in range(0,ncols):
                    row.append(0)
                row[i] = 1
                self.data.append(row)


    @staticmethod
    def fromDict(data):
        """
            Десеарилизация матрицы из словаря
        """
        ncols = data["ncols"]
        nrows = data["nrows"]
        items = data["data"]
        assert len(items) == ncols*nrows
        m = Matrix(nrows, ncols)
        for row in range(nrows):
            for col in range(ncols):
                m[(row, col)] = items[ncols*row + col]
        return m


    @staticmethod
    def toDict(M):
        """
            Сериализация матрицы в словарь
        """
        assert isinstance(M, Matrix)
        nrows, ncols = M.shape()
        data = []
        for row in range(nrows):
            for col in range(ncols):
                data.append(M[(row, col)])
        return {"nrows": nrows, "ncols": ncols, "data": data}


    def __str__(self):
        "Строковое представление матрицы"
        return '\n'.join(' '.join(map(str, row)) for row in self.data)


    def __repr__(self):
        return  '%s' % (self.data)


    def shape(self):
        "Вернуть кортеж размера матрицы (nrows, ncols)"
        return (self.nrows,self.ncols)


    def __getitem__(self, index):
        "Вернуть элемент с индесом index (кортеж (row, col))"
        row, col = index
        return self.data[row][col]


    def __setitem__(self, index, value):
        "Присвоить значение value элементу с индесом index (кортеж (row, col))"
        check_list = isinstance(index, list)
        check_tuple = isinstance(index, tuple)
        if not (check_list or check_tuple):
            raise ValueError('index is not list or typle.')
        if len(index)!=2:
            raise ValueError('len index must by 2')
        if self.nrows < index[0] or self.ncols < index[1]:
            raise IndexError('IndexError')
        row, col = index
        self.data[row][col] = value


    def __sub__(self, rhs):
        "Вычесть матрицу rhs и вернуть результат"
        if self.nrows != rhs.nrows or self.ncols != rhs.ncols:
            raise ValueError('Wrong size')

        for i in range(self.nrows):
            for j in range(self.ncols):
                self.data[i][j] = self.data[i][j] - rhs.data[i][j]
        return self


    def __add__(self, rhs):
        "Сложить с матрицей rhs и вернуть результат"
        if self.nrows != rhs.nrows or self.ncols != rhs.ncols:
            raise ValueError('Wrong size')

        for i in range(self.nrows):
            for j in range(self.ncols):
                self.data[i][j] = self.data[i][j] + rhs.data[i][j]
        return self


    def __mul__(self, rhs):
        "Умножить на матрицу rhs и вернуть результат"
        if self.nrows != rhs.ncols:
            raise ValueError('Wrong size')

        res = Matrix(self.nrows, rhs.ncols)
        for i in range(self.nrows):
            for j in range (rhs.ncols):
                summ = 0
                for k in range(self.ncols):
                    summ+=self.data[i][k]*rhs.data[k][j]
                res[(i,j)] = summ
        return res


    def __pow__(self, power):
        "Возвести все элементы в степень pow и вернуть результат"
        for i in range(self.nrows):
            for j in range(self.ncols):
                self.data[i][j] = self.data[i][j]**power
        return self


    def sum(self):
        "Вернуть сумму всех элементов матрицы"
        summ = 0
        for i in self.data:
            summ+=sum(i)
        return summ


    def __minor(self,i,j):
        res = Matrix(self.ncols-1,self.nrows-1)
        #print(res)
        for row in range(self.nrows):
            for col in range(self.ncols):
                #print(row,col)
                if(row != i and col != j):
                    ind_row = row if row < i else row -1
                    ind_col = col if col < j else col -1
                    res[(ind_row,ind_col)] = self.data[row][col]
        return res


    def det(self):
        "Вычислить определитель матрицы"
        if len(self.data) == 2:
            return self.data[0][0]*self.data[1][1]-self.data[0][1]*self.data[1][0]
        determinant = 0
        for i in range(0,len(self.data)):
            determinant+=(-1)**i*self.data[0][i]*(self.__minor(0,i)).det()
        return determinant


    def transpose(self):
        "Транспонировать матрицу и вернуть результат"
        res = Matrix(self.ncols,self.nrows)
        for i in range(self.ncols):
            for j in range(self.nrows):
                res[(i,j)] = self.data[j][i]
        return res


    def inv(self):
        "Вычислить обратную матрицу и вернуть результат"
        if self.ncols != self.nrows:
            raise ArithmeticError('матрица не квадратная')

        res = Matrix(self.ncols,self.nrows)

        determinant = self.det()
        #special case for 2x2 matrix:
        if len(self.data) == 2:
            res[(0,0)] = self.data[1][1]/determinant
            res[(0,1)] = -1*self.data[0][1]/determinant
            res[(1,0)] = -1*self.data[1][0]/determinant
            res[(1,1)] = self.data[0][0]/determinant
            return res

        cofactors = []
        for row in range(len(self.data)):
            cofactorRow = []
            for col in range(len(self.data)):
                minor = self.__minor(row,col)
                cofactorRow.append(((-1)**(row+col)) * minor.det())
            cofactors.append(cofactorRow)
        for row in range(len(self.data)):
            for col in range(len(self.data)):
                res[(row,col)] = cofactors[row][col]
        res = res.transpose()
        for row in range(len(self.data)):
            for col in range(len(self.data)):
                res[(row,col)] = res[(row,col)]/determinant
        return res


    def tonumpy(self):
        "Приведение к массиву numpy"
        return np.array(self.data)


    def raund(self):
        "raund"
        for i in range(len(self.data)):
            for j in range(len(self.data)):
                self.data[i][j] = round(self.data[i][j], 3)
        return self




In [3]:
def load_file(filename):
    with open(filename, "r") as f:
        input_file = json.load(f)
        A = Matrix.fromDict(input_file["A"])
        B = Matrix.fromDict(input_file["B"])
    return A, B

In [4]:
# Задайте в filename путь к файлу, полученному от бота
filename = "input_085.json"
A, B = load_file(filename)

Проверка реализованных методов

In [5]:
# Задайте в filename путь к файлу, полученному от бота
filename = "input_085.json"
A, B = load_file(filename)
print("Матрица A: ")
print(A)
print("Матрица B: ")
print(B)
C = A*B
print("Матрица C = A*B: ")
print(C)
C_t = C.transpose()
print("Транспонированная матрица C: ")
print(C_t)


C_inv = C.inv()
print("Матрица, обратная C: ")
print(C_inv)
E = Matrix(C_inv.ncols, C_inv.nrows, init="eye")
D = C_inv + E
print("Матрица D равная сумме C и единичной матрицы: ")
print(D)
D_det = D.det()
print("Определитель матрицы D: ", D_det)
D_norm = (D**2).sum()**0.5
print("Норма Фробениуса матрицы D: ", D_norm)

Матрица A: 
0.9373077709673506 0.7002470682891963 0.2649731112315512 0.7883239988145216
0.7185363804667407 0.7096638195779915 0.7446544852001051 0.08922339485981134
0.5790212653361246 0.28342529054150045 0.7911378431230159 0.4164620623492461
Матрица B: 
0.31882529377780333 0.33032709542769656 0.48551547856311295
0.032384219874779685 0.4644639246573763 0.896453439825196
0.30857402710659587 0.20162311924594611 0.7348082465444719
0.8813738186797379 0.4968830512548078 0.40564223710390346
Матрица C = A*B: 
1.0980863336647506 1.0799871941840349 1.5972982617648037
0.5604896793070852 0.761438431095032 1.5684121408517988
0.804968480428244 0.6893518555173499 1.2854705772977235
Транспонированная матрица C: 
1.0980863336647506 0.5604896793070852 0.804968480428244
1.0799871941840349 0.761438431095032 0.6893518555173499
1.5972982617648037 1.5684121408517988 1.2854705772977235
Матрица, обратная C: 
-0.9216996870254355 -2.58547745473848 4.299844458116418
4.879691480717298 1.1323775017309523 -7.4450224

Сохранение результатов в файл. Не изменяйте этот код. Отправтье файл result.json в ответ на это задание.

In [6]:
A_dict = Matrix.toDict(A)
B_dict = Matrix.toDict(B)
C_dict = Matrix.toDict(C)
Ct_dict = Matrix.toDict(C_t)
Cinv_dict = Matrix.toDict(C_inv)
result = {
    "A": A_dict,
    "B": B_dict,
    "C": C_dict,
    "Ct": Ct_dict,
    "Cinv": Cinv_dict,
    "D_det": D_det,
    "D_norm": D_norm
}

In [8]:
def save_file(filename, data):
    with open(filename, "w") as f:
        input_file = json.dump(data, f)

In [9]:
save_file("result.json", result)